In [1]:
import requests as req
from bs4 import BeautifulSoup as bsp
import pandas as pd
import time
from selenium import webdriver
import requests
from selenium.webdriver.edge.service import Service as EdgeService
from selenium.webdriver.edge.options import Options as EdgeOptions

In [2]:
info = {  # uninitialized dictionary to use for later
        'Name': [],
        'Price': [],
        'Image': [],
        'Link':[]
    }

In [3]:
url = 'https://pk.sapphireonline.pk/'

In [4]:
# Getting the navbar of the website
soup = bsp(req.get(url).content, 'html.parser')
# getting the navbar of the website which has a ul tag with class 't4s-nav__ul t4s-d-inline-flex t4s-flex-wrap t4s-align-items-center'
navbar = soup.find('ul', class_='t4s-nav__ul t4s-d-inline-flex t4s-flex-wrap t4s-align-items-center')
# In the ul there are li tag each tag has a href in the a tag
links = [link.find('a')['href'] for link in navbar.find_all('li')]
# adding the url to the links
links = [url + link for link in links]

for link in links:
    print(link)

https://pk.sapphireonline.pk//collections/new-in
https://pk.sapphireonline.pk//collections/woman
https://pk.sapphireonline.pk//collections/man
https://pk.sapphireonline.pk//collections/kids
https://pk.sapphireonline.pk//collections/beauty
https://pk.sapphireonline.pk//collections/accessories
https://pk.sapphireonline.pk//collections/home
https://pk.sapphireonline.pk//pages/the-edit 


In [5]:
def get_valid_proxies():
    proxy_list_url = 'https://free-proxy-list.net/'
    response = requests.get(proxy_list_url)
    soup = bsp(response.text, 'html.parser')
    proxy_data = []
    rows = soup.find_all('tr')[1:]
    for row in rows:
        columns = row.find_all('td')
        if len(columns) >= 8:
            ip_address = columns[0].text.strip()
            google_enabled = columns[5].text.strip().lower() == 'yes'
            https_enabled = columns[6].text.strip().lower() == 'yes'
            last_checked = columns[7].text.strip()
            if (last_checked.endswith('mins ago') and int(last_checked.split(' ')[0]) < 15) or last_checked.endswith('hours ago'):
                if google_enabled or https_enabled:
                    proxy_data.append({'ip_address': ip_address, 'google_enabled': google_enabled, 'https_enabled': https_enabled})

    return proxy_data

def rotate_user_agent(proxy):
    if proxy:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3',
            'http': f'http://{proxy}',
            'https': f'https://{proxy}'
        }
    else:
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
        }
    return headers

In [6]:
# Scrapping the links one by one
for link in links:
    # Getting the navbar of the website
    response = req.get(link)
    if not response.ok:
        print("Server responded with exit code:", response.status_code) # if scrapping is not allowed
    else:
        soup = bsp(response.content, 'html.parser')
        # scrolling the page to the bottom above the footer then back up to load all the items until the end
        edge_options = EdgeOptions()
        edge_options.use_chromium = True
        edge_options.add_argument("--start-maximized")
        driver = webdriver.Edge(service=EdgeService(), options=edge_options)
        driver.get(link)
        Previous_Height = driver.execute_script("return document.body.scrollHeight")
        while True:
            # go to the bottom of the page to load all the items
            driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
            time.sleep(2) # wait for 2 seconds
            New_Height = driver.execute_script("return document.body.scrollHeight")
            if New_Height == Previous_Height:
                break
            Previous_Height = New_Height
            


        # getting the html content of the page
        html = driver.page_source
        driver.quit()
        # parsing the html content
        soup = bsp(html, 'html.parser')
        pretty = soup.prettify() # increasing readability
        with open('scrapped.html', 'w', encoding='utf-8') as htmlFile:  # specify encoding method
            htmlFile.write(pretty)


        # getting the div with class 't4s-section-inner t4s_nt_se_template--16016591585354__main t4s_se_template--16016591585354__main t4s-container-fluid'
        products = soup.find('div', class_='t4s-section-inner t4s_nt_se_template--16016591585354__main t4s_se_template--16016591585354__main t4s-container-fluid').find('div', class_='t4s-row').find_all('div', class_='t4s-product t4s-pr-grid t4s-pr-style1 t4s-pr-7738123616330 t4s-col-item is-t4s-pr-created')
        
        for product in products:
            # getting the href from div with class 't4s-product-btns t4s-col-2 t4s-col-lg-5'
            link = product.find('div', class_='t4s-product-btns t4s-col-2 t4s-col-lg-5').find('a')['href']
            # getting name in h3 tag of class 't4s-product-title'
            name = product.find('h3', class_='t4s-product-title').text.strip()
            # getting price from div with class 't4s-product-price'
            price = product.find('div', class_='t4s-product-price').text.strip()
            # getting image from div with class 't4s-product-img t4s_ratio is-show-img2'
            image = product.find('div', class_='t4s-product-img t4s_ratio is-show-img2').find('img')['src']
            
            info['Name'].append(name)
            info['Price'].append(price)
            info['Image'].append(url+image)
            info['Link'].append(url+link)

AttributeError: 'NoneType' object has no attribute 'find'

In [7]:
print(info)

{'Name': [], 'Price': [], 'Image': [], 'Link': []}


In [ ]:
# Write dictionary to CSV file
df = pd.DataFrame(info)
df.to_csv('Sapphire.csv')